In [1]:
from davitpy import pydarn
import datetime
import numpy
import pandas
import dask
import time

In [2]:
startTime = datetime.datetime(2017,12,2)
endTime = datetime.datetime(2017,12,3)
radSel = 'bks'
fileType = 'fitacf3'
filtered = False

In [3]:
f = pydarn.sdio.radDataOpen(startTime, radSel,
                    endTime, filtered=filtered,
                    fileType=fileType)

In [4]:
# myBeam = pydarn.sdio.radDataReadRec(f)
# print myBeam.recordDict.keys()

In [5]:
# def test_dask(m, qflgArr, gateArr, velArr, spwArr, pwrArr, gflgArr, dtArr, bmArr):
#     qflgArr = numpy.concatenate( ( qflgArr, numpy.array(m.fit.qflg ) ) )
#     gateArr = numpy.concatenate( ( gateArr, numpy.array(m.fit.slist ) ) )
#     velArr = numpy.concatenate( ( velArr, numpy.array(m.fit.v ) ) )
#     spwArr = numpy.concatenate( ( spwArr, numpy.array(m.fit.w_l ) ) )
#     pwrArr = numpy.concatenate( ( pwrArr, numpy.array(m.fit.p_l ) ) )
#     gflgArr = numpy.concatenate( ( gflgArr, numpy.array(m.fit.gflg ) ) )
#     # date and bmnum are a little different
#     dtArr = numpy.concatenate( ( dtArr, numpy.full( (len(m.fit.qflg)), m.time) ) )
#     bmArr = numpy.concatenate( ( bmArr, numpy.full( (len(m.fit.qflg)), m.bmnum) ) )
#     print bmArr.shape
#     return 1

In [6]:
# mb = pydarn.sdio.radDataReadAll(f)

In [7]:
stest1 = time.time()
x = []
# we'll create a dataframe with all the data
# To keep things simple (at this moment), I'm
# not using multi-indexed dataframe.
# Initialize empty arrays for the data
# these will later be converted to a DF
qflgArr = numpy.empty([0])
gateArr = numpy.empty([0])
velArr = numpy.empty([0])
spwArr = numpy.empty([0])
pwrArr = numpy.empty([0])
gflgArr = numpy.empty([0])
dtArr = numpy.empty([0])
bmArr = numpy.empty([0])
for n, m in enumerate(mb):
#     print numpy.array(m.fit.qflg )
    if m is None:
        continue
    if m.fit is None:
        continue
    if m.fit.v is None:
        continue
    qflgArr = numpy.concatenate( ( qflgArr, numpy.array(m.fit.qflg ) ) )
    gateArr = numpy.concatenate( ( gateArr, numpy.array(m.fit.slist ) ) )
    velArr = numpy.concatenate( ( velArr, numpy.array(m.fit.v ) ) )
    spwArr = numpy.concatenate( ( spwArr, numpy.array(m.fit.w_l ) ) )
    pwrArr = numpy.concatenate( ( pwrArr, numpy.array(m.fit.p_l ) ) )
    gflgArr = numpy.concatenate( ( gflgArr, numpy.array(m.fit.gflg ) ) )
    # date and bmnum are a little different
    dtArr = numpy.concatenate( ( dtArr, numpy.full( (len(m.fit.qflg)), m.time) ) )
    bmArr = numpy.concatenate( ( bmArr, numpy.full( (len(m.fit.qflg)), m.bmnum) ) )
dfx = dask.delayed(pandas.DataFrame( {
    "qflg" : qflgArr,
    "gate" : gateArr,
    "vel" : velArr,
    "spw" : spwArr,
    "pwr" : pwrArr,
    "gflg" : gflgArr,
    "date" : dtArr,
    "beam" : bmArr,
})).compute()
# Convert int cols to int16
# they are being stored as 
# floats now! save space!
print "time cons1--->", time.time()-stest1
intCols = [ "qflg", "gate", "gflg", "beam" ]
dfx[intCols] = dfx[intCols].astype(numpy.int16)
print qflgArr.compute()

NameError: name 'mb' is not defined

In [ ]:
dfx.info(memory_usage=True)

In [ ]:
print dfx.head()
print "------------------"
print dfx.dtypes

In [ ]:
# print mb[0].fit.qflg
# print "---------------------------"
# # print mb[0].fit.slist
# # print "---------------------------"
# # print mb[0].fit.v
# # print "---------------------------"
# # print mb[0].fit.w_l
# # print "---------------------------"
# # print mb[0].fit.p_l
# # print "---------------------------"
# # print mb[0].fit.gflg
# # print "---------------------------"
# # print mb[0].time
# # print "---------------------------"
# # print numpy.full( (len(mb[0].fit.qflg)), mb[0].bmnum).shape
# print numpy.array(mb[0].fit.slist )